In [1]:
from google.colab import auth


auth.authenticate_user()


project_id = 'ee-nssurya21'
!gcloud config set project {project_id}


Updated property [core/project].


In [2]:
# Install geemap library
!pip install geemap

# Import necessary libraries
import ee
import geemap
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.1 MB/s eta 0:00:00


In [3]:
# Authenticate to Earth Engine
try:
    # Attempt to initialize Earth Engine with the specified project ID
    ee.Initialize(project= project_id)  # Replace with your actual project ID

    # If initialization is successful, no exceptions are raised, and you can proceed with Earth Engine code
    print("Earth Engine initialized successfully!")

except ee.EEException as e:
    # If initialization fails, an exception is raised, and authentication is needed
    print("Authentication required. Initiating authentication...")

    # Authenticate to Earth Engine
    ee.Authenticate()

    # Retry initialization after authentication
    ee.Initialize(project=project_id)  # Replace with your actual project ID

    print("Earth Engine initialized successfully after authentication!")


Earth Engine initialized successfully!


In [4]:
import folium

In [5]:
# Define the region of interest (ROI) for Ranthambore
roi_ranthambore = ee.Geometry.Point([76.5000, 26.0167])

# Load a Landsat image collection for a specific time and location (using Ranthambore ROI)
landsat_collection_ranthambore = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
                                   .filterBounds(roi_ranthambore)
                                   .filterDate('2021-01-01', '2021-12-31')
                                   .sort('system:time_start'))

# Function to calculate NDVI
def calculate_ndvi(image):
    return image.normalizedDifference(['B5', 'B4']).rename('NDVI')


# Create a folium map centered around the Ranthambore ROI
map_center_ranthambore = [roi_ranthambore.getInfo()['coordinates'][1], roi_ranthambore.getInfo()['coordinates'][0]]
m_ranthambore = folium.Map(location=map_center_ranthambore, zoom_start=10)

# Add a marker for the Ranthambore ROI
folium.Marker(map_center_ranthambore, popup='Ranthambore ROI').add_to(m_ranthambore)

# Display the folium map for Ranthambore
display(m_ranthambore)


In [6]:
# Define the region of interest
import pandas as pd
roi = ee.Geometry.Point([76.5000, 26.0167])

# Load the image collection
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2014-01-01', '2014-12-31').filterBounds(roi)

# Initialize an empty DataFrame to store the NDVI values
ndvi_df = pd.DataFrame(columns=['image_id', 'ndvi'])

# Iterate over the image collection
for i in range(image_collection.size().getInfo()):
    # Get the image
    image = ee.Image(image_collection.toList(image_collection.size()).get(i))

    # Select the red and near-infrared bands
    red = image.select('B4')
    nir = image.select('B5')

    # Calculate NDVI
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

    # Get the mean NDVI value for the region of interest
    ndvi_value = ndvi.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30).getInfo()['NDVI']

    # Get the image ID
    image_id = image.get('system:index').getInfo()

    # Store the NDVI value in the DataFrame
    ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)

# Print the DataFrame

print(ndvi_df)

<ipython-input-6-ea292aa5833b>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)
<ipython-input-6-ea292aa5833b>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)
<ipython-input-6-ea292aa5833b>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)
<ipython-input-6-ea292aa5833b>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': 

                image_id      ndvi
0   LC08_146042_20140108  0.563304
1   LC08_146042_20140124  0.514580
2   LC08_146042_20140209  0.481315
3   LC08_146042_20140225  0.219496
4   LC08_146042_20140313  0.396445
5   LC08_146042_20140329  0.398972
6   LC08_146042_20140414  0.312276
7   LC08_146042_20140516  0.390635
8   LC08_146042_20140601  0.368625
9   LC08_146042_20140617  0.409004
10  LC08_146042_20140703  0.119014
11  LC08_146042_20140719  0.063383
12  LC08_146042_20140820  0.752882
13  LC08_146042_20140905  0.036300
14  LC08_146042_20140921  0.780409
15  LC08_146042_20141007  0.731495
16  LC08_146042_20141023  0.687741
17  LC08_146042_20141108  0.671216
18  LC08_146042_20141124  0.635845
19  LC08_146042_20141210  0.616260
20  LC08_146042_20141226  0.555707
21  LC08_147042_20140115  0.490588
22  LC08_147042_20140131  0.496276
23  LC08_147042_20140216  0.002908
24  LC08_147042_20140320  0.406082
25  LC08_147042_20140405  0.359671
26  LC08_147042_20140421  0.398539
27  LC08_147042_2014

<ipython-input-6-ea292aa5833b>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)


In [7]:

# Define the region of interest
roi = ee.Geometry.Point([-122.0818, 37.4203])

# Load a Landsat 8 image
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).first()
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2014-01-01', '2014-12-31').filterBounds(roi)
# Select the red and near-infrared bands
red = image.select('B4')
nir = image.select('B5')

# Calculate NDVI
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

# Print the NDVI value
ndvi_info = ndvi.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30).getInfo()
print('NDVI:', ndvi_info['NDVI'])
print(type(ndvi))
print(type(ndvi_info))

NDVI: 0.6493473253135398
<class 'ee.image.Image'>
<class 'dict'>


In [8]:
# Sort the collection by 'system:time_start'
image_collection = image_collection.sort('system:time_start')

# Get the ID of the second image
second_image_id = ee.Image(image_collection.toList(image_collection.size()).get(1)).get('system:index').getInfo()

# Filter the collection to get the second image
second_image = image_collection.filter(ee.Filter.eq('system:index', second_image_id))

# Get information about the second image
second_image_info = second_image.first().getInfo()

# Print the information
print(second_image_info)

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 461085, 0, -30, 4264215]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 461085, 0, -30, 4264215]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 461085, 0, -30, 4264215]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 461085, 0, -30, 4264215]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 4610

In [9]:
# Define the region of interest
import pandas as pd
roi = ee.Geometry.Point([-122.0818, 37.4203])

# Load the image collection
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2013-01-01', '2014-12-31').filterBounds(roi)

# Initialize an empty DataFrame to store the NDVI values
ndvi_df = pd.DataFrame(columns=['image_id', 'ndvi'])

# Iterate over the image collection
for i in range(image_collection.size().getInfo()):
    # Get the image
    image = ee.Image(image_collection.toList(image_collection.size()).get(i))

    # Select the red and near-infrared bands
    red = image.select('B4')
    nir = image.select('B5')

    # Calculate NDVI
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

    # Get the mean NDVI value for the region of interest
    ndvi_value = ndvi.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30).getInfo()['NDVI']

    # Get the image ID
    image_id = image.get('system:index').getInfo()

    # Store the NDVI value in the DataFrame
    ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)

# Print the DataFrame
print(ndvi_df)

<ipython-input-9-555d4ea9d758>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)
<ipython-input-9-555d4ea9d758>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)
<ipython-input-9-555d4ea9d758>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)
<ipython-input-9-555d4ea9d758>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': 

                image_id      ndvi
0   LC08_044034_20130409  0.649347
1   LC08_044034_20130416  0.611197
2   LC08_044034_20130603  0.562339
3   LC08_044034_20130619  0.576914
4   LC08_044034_20130705  0.622691
5   LC08_044034_20130721  0.617317
6   LC08_044034_20130806  0.575530
7   LC08_044034_20130822  0.599776
8   LC08_044034_20130907  0.668696
9   LC08_044034_20130923  0.713694
10  LC08_044034_20131009  0.670517
11  LC08_044034_20131025  0.675733
12  LC08_044034_20131110  0.627347
13  LC08_044034_20131126  0.645191
14  LC08_044034_20131212  0.681148
15  LC08_044034_20131228  0.607627
16  LC08_044034_20140113  0.606061
17  LC08_044034_20140129  0.029374
18  LC08_044034_20140214  0.029597
19  LC08_044034_20140318  0.645625
20  LC08_044034_20140403  0.333333
21  LC08_044034_20140419  0.652893
22  LC08_044034_20140505  0.063379
23  LC08_044034_20140521  0.635908
24  LC08_044034_20140606  0.640540
25  LC08_044034_20140622  0.612822
26  LC08_044034_20140708  0.622019
27  LC08_044034_2014

<ipython-input-9-555d4ea9d758>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndvi_df = ndvi_df.append({'image_id': image_id, 'ndvi': ndvi_value}, ignore_index=True)


In [10]:
ndvi_df.shape

(34, 2)

In [11]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Define a region of interest.
roi = ee.Geometry.Polygon(
  [[[76.842,11.481],
    [77.249,11.481],
    [77.249,10.935],
    [76.842,10.935],
    [76.842,11.481]]]
)

# Filter the L8 collection to a single date.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(roi)

# Make a cloud-free composite.
composite = ee.Algorithms.Landsat.simpleComposite(collection)

# Calculate NDVI.
ndvi = composite.normalizedDifference(['B5', 'B4']).rename('NDVI')

# Clip the NDVI to the ROI.
clipped = ndvi.clip(roi)

# Calculate the mean NDVI for the entire image.
mean_ndvi = clipped.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30, maxPixels=1e9)

# Print the result to the console.
print('Mean NDVI:', mean_ndvi.getInfo()['NDVI'])

# Define a sub-region of interest within the clipped image.
sub_roi = ee.Geometry.Rectangle([76.9, 11.2, 77.0, 11.1])

# Calculate the mean NDVI within the sub-region.
mean_ndvi_sub = clipped.reduceRegion(reducer=ee.Reducer.mean(), geometry=sub_roi, scale=30, maxPixels=1e9)

# Print the result to the console.
print('Mean NDVI in sub-region:', mean_ndvi_sub.getInfo()['NDVI'])

Mean NDVI: 0.41911560093170225
Mean NDVI in sub-region: 0.4183791543364513


In [12]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Define a region of interest.
roi = ee.Geometry.Rectangle([80.18, 12.80, 80.27, 13.04])

# Filter the L8 collection to a single date.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(roi)

# Make a cloud-free composite.
composite = ee.Algorithms.Landsat.simpleComposite(collection)

# Calculate NDVI.
ndvi = composite.normalizedDifference(['B5', 'B4']).rename('NDVI')

# Clip the NDVI to the ROI.
clipped = ndvi.clip(roi)

# Calculate the mean NDVI for the entire image.
mean_ndvi = clipped.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30, maxPixels=1e9)

# Print the result to the console.
print('Mean NDVI:', mean_ndvi.getInfo()['NDVI'])

# Define a sub-region of interest within the clipped image.
sub_roi = ee.Geometry.Rectangle([76.9, 11.2, 77.0, 11.1])

# Calculate the mean NDVI within the sub-region.
mean_ndvi_sub = clipped.reduceRegion(reducer=ee.Reducer.mean(), geometry=sub_roi, scale=30, maxPixels=1e9)

# Print the result to the console.
print('Mean NDVI in sub-region:', mean_ndvi_sub.getInfo()['NDVI'])

Mean NDVI: 0.251802540435411
Mean NDVI in sub-region: None


In [18]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Define a region of interest.
roi = ee.Geometry.Rectangle([76.842, 11.481, 77.249, 10.935])

# Filter the L8 collection to a single date.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterDate('2020-02-01', '2020-03-31') \
    .filterBounds(roi)

# Get the list of images in the collection.
images = collection.toList(collection.size())

# Loop over the images and create an export task for each one.
for i in range(images.size().getInfo()):
    image = ee.Image(images.get(i))
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    clipped = ndvi.clip(roi)
    task = ee.batch.Export.image.toDrive(
        image=clipped,
        description='NDVI_' + image.id().getInfo(),
        folder='Projects',
        scale=30,
        region=roi.getInfo()['coordinates']
    )
    task.start()
    print(f'Started export task for image {image.id().getInfo()}')

Started export task for image LC08_144052_20200212
Started export task for image LC08_144052_20200228
Started export task for image LC08_144052_20200315
Started export task for image LC08_144053_20200212
Started export task for image LC08_144053_20200228
Started export task for image LC08_144053_20200315


In [20]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Define a region of interest.
roi = ee.Geometry.Polygon(
  [[[76.842,11.481],
    [77.249,11.481],
    [77.249,10.935],
    [76.842,10.935],
    [76.842,11.481]]
  ]
)

# Filter the L8 collection to a single date.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterDate('2020-01-01', '2020-02-29') \
    .filterBounds(roi)

# Make a cloud-free composite.
composite = ee.Algorithms.Landsat.simpleComposite(collection)

# Calculate NDVI.
ndvi = composite.normalizedDifference(['B5', 'B4']).rename('NDVI')

# Clip the NDVI to the ROI.
clipped = ndvi.clip(roi)

# Export the NDVI to your drive.
task = ee.batch.Export.image.toDrive(
    image=clipped,
    description='NDVI_Coimbatore_2',
    folder='Projects',
    scale=30,
    region=roi.getInfo()['coordinates'],
    maxPixels=1e9
)
task.start()